In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [2]:
torch.manual_seed(42)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
train = pd.read_csv('/content/fashion-mnist_train.csv')
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
train.shape

(60000, 785)

In [6]:
test = pd.read_csv('/content/fashion-mnist_test.csv')
test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
test.shape

(10000, 785)

In [8]:
X_train = train.drop('label', axis=1).values
y_train = train['label'].values
X_test = test.drop('label', axis=1).values
y_test = test['label'].values

In [9]:
X_train = X_train/255.0
X_test = X_test/255.0

In [10]:
class FashionD(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]

In [11]:
train_dataset = FashionD(X_train, y_train)
test_dataset = FashionD(X_test, y_test)

In [12]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [13]:
class FashionMNIST(nn.Module):
  def __init__(self, num_features):
    super(FashionMNIST, self).__init__()

    self.model = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
    )

  def forward(self, x):
    return self.model(x)

In [14]:
learning_rate = 0.1
epochs = 100

In [15]:
model = FashionMNIST(X_train.shape[1]).to(device)

loss_function = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [19]:
for epoch in range(epochs):
  total_loss = 0

  for batch_features, batch_labels in train_loader:
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)

    outputs = model(batch_features)

    loss = loss_function(outputs, batch_labels)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch+1}/{epoch}, Loss: {total_loss/len(train_loader)}")

Epoch 1/0, Loss: 0.060080894933286694
Epoch 2/1, Loss: 0.060433316636514835
Epoch 3/2, Loss: 0.06351556296860958
Epoch 4/3, Loss: 0.05719774833389882
Epoch 5/4, Loss: 0.05592520250444875
Epoch 6/5, Loss: 0.05247912824705942
Epoch 7/6, Loss: 0.05762048718581068
Epoch 8/7, Loss: 0.05629422656992184
Epoch 9/8, Loss: 0.055403468254179175
Epoch 10/9, Loss: 0.0520173141406154
Epoch 11/10, Loss: 0.05478948042584039
Epoch 12/11, Loss: 0.05367604679756188
Epoch 13/12, Loss: 0.05247859202133222
Epoch 14/13, Loss: 0.04779096787595855
Epoch 15/14, Loss: 0.048726718500703335
Epoch 16/15, Loss: 0.04926821942749455
Epoch 17/16, Loss: 0.04907688162084033
Epoch 18/17, Loss: 0.04227554332142247
Epoch 19/18, Loss: 0.04808632976738915
Epoch 20/19, Loss: 0.046821833278652206
Epoch 21/20, Loss: 0.04325387303328101
Epoch 22/21, Loss: 0.04334438173919479
Epoch 23/22, Loss: 0.04645901644102999
Epoch 24/23, Loss: 0.04705437800466224
Epoch 25/24, Loss: 0.04116542718677557
Epoch 26/25, Loss: 0.04272642249132502
E

In [20]:
model.eval()

FashionMNIST(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [21]:
total = 0
correct = 0

with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total += batch_labels.shape[0]
    correct += (predicted == batch_labels).sum().item()

print(f"Accuracy: {100*correct/total: .2f}%")

Accuracy:  90.01%
